In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

## Data Set

In [2]:
x1 = np.array([ 0.        ,  3.33333333,  6.66666667, 10.        ,  0.        ,
        3.33333333,  6.66666667, 10.        ,  0.        ,  3.33333333,
        6.66666667, 10.        ,  0.        ,  3.33333333,  6.66666667,
       10.        ])
x2 = np.array([0.        , 0.        , 0.        , 0.        , 0.33333333,
       0.33333333, 0.33333333, 0.33333333, 0.66666667, 0.66666667,
       0.66666667, 0.66666667, 1.        , 1.        , 1.        ,
       1.        ])
y = np.array([ 1.22353179,  3.57574634,  5.89217837,  8.19889324,  1.96307308,
        4.57556858,  7.48364786,  9.2400334 ,  2.9048125 ,  5.74174653,
        8.19832673, 10.69424417,  4.15380028,  7.12275093,  8.8976015 ,
       11.76820779])

In [3]:
X = pd.DataFrame({'x1':x1, 'x2':x2})

## 1
a) Run a linear regression to find A, B1, and B2.  
b) Calculate SE(B1) and SE(B2)  
c) Calculate a prediction, y_hat, at x1=5, x2=.5 along with $SE(yhat) = \sqrt{RSS/(n-k-1)} / \sqrt{n}$

### a)

In [4]:
# add constant ones column to incorporate intercept
X = sm.add_constant(X)
regress = sm.OLS(y, X)
results = regress.fit()

In [5]:
print("A =", results.params[0])
print("B1 =", results.params[1])
print("B2 =", results.params[2])

A = 1.0324877889494957
B1 = 0.7381831868215653
B2 = 3.257462940135361


### b)

In [6]:
print("SE(B1) =", results.bse[1])
print("SE(B2) =", results.bse[2])

SE(B1) = 0.016217367403375465
SE(B2) = 0.16217367374184205


### c)

In [7]:
# add constant entry to match required dimension
yhat = results.predict([1, 5, 0.5])

print("y_hat =", yhat[0])

y_hat = 6.352135193125003


In [8]:
n = len(y)
k = 2
RSS = ((y - yhat)**2).sum()

SE_yhat = np.sqrt(RSS / (n - k - 1)) / np.sqrt(n)

print("SE(yhat) =", SE_yhat)

SE(yhat) = 0.8361772658727425


## 2
a) Calculate m=100 bootstrap sample sets from the original set of 16 samples and, for each bootstrap sample set, estimate A_bs, B1_bs, and B2_bs by regression. (You'll have 100 estimates of each of A_bs, B1_bs, and B2_bs.)  
b) Compute SE(B1_bs) and SE(B2_bs) directly from the m=100 samples. Do they match SE(B1) and SE(B2) from part 1, above?  
c) For each model, A_bs, B1_bs, and B2_bs, calculate a prediction, y_hat_bs, at x1=5, x2=.5. From that set of predictions, compute SE(y_hat_bs). Does it match SE(y_hat) from part 1, above?  
  
  
Hint: Be sure the values of y, x1, and x2 stay aligned when you draw a sample from the original data. Ex: Randomly drawing the i'th sample would yield (y[i],x1[i],x2[i]) -- all with the same i.

In [9]:
def bootstrap(m, x1, x2, y):
    A_bs = np.zeros(m)
    B1_bs = np.zeros(m)
    B2_bs = np.zeros(m)
    yhat = np.zeros(m)
    n = len(y)
    
    # create m bootstrapped simulations
    for i in range(m):
        x1_bs = np.zeros(n)
        x2_bs = np.zeros(n)
        y_bs = np.zeros(n)
        
        # create bootstrapped data sample
        # randomly pick an index to keep data aligned
        for j in range(n):
            s = np.random.choice(range(n))
            x1_bs[j] = x1[s]
            x2_bs[j] = x2[s]
            y_bs[j] = y[s]
        
        X_bs = pd.DataFrame({'x1':x1_bs, 'x2':x2_bs})
        X_bs = sm.add_constant(X_bs)
        
        # run bootstrapped sample regression
        regress_bs = sm.OLS(y_bs, X_bs)
        results_bs = regress_bs.fit()
        
        A_bs[i] = results_bs.params[0]
        B1_bs[i] = results_bs.params[1]
        B2_bs[i] = results_bs.params[2]
        yhat[i] = (results_bs.predict([1, 5, 0.5]))[0]
    
    return A_bs, B1_bs, B2_bs, yhat

### a)

In [10]:
A_bs, B1_bs, B2_bs, yhat_bs = bootstrap(100, x1, x2, y)

In [11]:
print("Bootstrapped A =", A_bs.mean())
print("Bootstrapped B1 =", B1_bs.mean())
print("Bootstrapped B2 =", B2_bs.mean())

Bootstrapped A = 1.0279436384560965
Bootstrapped B1 = 0.7416140167517391
Bootstrapped B2 = 3.2483165442877686


### b)

In [12]:
print("Bootstrapped SE(B1) =", B1_bs.std())
print("Bootstrapped SE(B2) =", B2_bs.std())

Bootstrapped SE(B1) = 0.017247192937962653
Bootstrapped SE(B2) = 0.16389826614066508


The standard errors of the regression coefficients for the bootstrapped data are very similar to those seen in the original regression.

### c)

In [13]:
print("Bootstrapped y_hat =", yhat_bs.mean())

Bootstrapped y_hat = 6.3601719943586765


In [14]:
print("Bootstrapped SE(yhat) =", yhat_bs.std())

Bootstrapped SE(yhat) = 0.06530748463845208


The $SE(\hat{Y})$ for the bootstrapped data ensemble is much smaller than it was for the original regression.